In [2]:
# Import packages

import os
import torch
import numpy as np
import cv2
from speciesnet import DEFAULT_MODEL
from speciesnet import draw_bboxes
from speciesnet import load_rgb_image
from speciesnet import SpeciesNet
from speciesnet import SpeciesNetClassifier
from speciesnet import SUPPORTED_MODELS
from megadetector.visualization import visualization_utils as vis_utils
from megadetector.detection import run_detector
from helper_functions import *
from inference_functions import *
import torchvision.transforms.functional as F
from PIL import Image
from yolo_detector import yolo_inference
%matplotlib inline

/opt/anaconda3/envs/ecobio/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/ecobio/lib/python3.10/site-packages/yolov5/utils/general.py:34: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg


In [3]:
# Load models

custom_megadetector_model = run_detector.load_detector('./models/md_v5a.0.0.pt')
species_net = SpeciesNet('./models')
species_net_classifier_model = torch.load('./models/always_crop_99710272_22x8_v12_epoch_00148.pt', weights_only=False)
species_net_classifier_model.eval()
with open('./models/always_crop_99710272_22x8_v12_epoch_00148.labels.txt', mode="r", encoding="utf-8") as fp:
  labels = {idx: line.strip() for idx, line in enumerate(fp.readlines())}
  
with open('./models/species_dict.json', 'r') as species_dict:
  species_dict = json.load(species_dict)

Loading PT detector with compatibility mode classic


Fusing layers... 
Model summary: 575 layers, 140045044 parameters, 0 gradients, 208.8 GFLOPs


In [4]:
# Load Detector and Classifier

yolo = yolo_inference(
        detector_model=custom_megadetector_model,
        classifier_model = species_net,
        file_name = 'Zorrillo 5.AVI',
        file_extension = 'AVI',
        steps = 30,
        find_n_frames = 5
        )


In [ ]:
# Pipeline

yolo_metadata = yolo.detect_and_predict_image()
speciesnet_preds = species_net_to_cupybara(yolo_metadata, species_dict)
final_predict(yolo_metadata, speciesnet_preds)